#### Практическое задание к лекции №3

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k

ImportError: cannot import name 'precision_at_k' from 'metrics' (c:\Users\azhur\AppData\Local\Programs\Python\Python310\lib\site-packages\metrics\__init__.py)

In [5]:
data = pd.read_csv('../webinar_3/data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '../webinar_3/data/transaction_data.csv'

item_features = pd.read_csv('../webinar_3/data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

In [ ]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [ ]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], # будем отфильтровывать товары не из топа
                                    recalculate_user=True)]
    return res

In [ ]:
def als_fit(user_item_matrix, params, result):
    model = AlternatingLeastSquares(**params)
    model.fit(csr_matrix(user_item_matrix).T.tocsr())
    result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
    score = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    return score

Реализуем grid search (код взят отсюда:
https://codereview.stackexchange.com/questions/171173/list-all-possible-permutations-from-a-python-dictionary-of-lists
https://www.kaggle.com/willkoehrsen/intro-to-model-tuning-grid-and-random-search)

In [ ]:
import itertools

In [ ]:
def grid_search(user_item_matrix, param_grid, result):
    simile=lambda x: x
    score=[]
    for weight, method in zip([simile, tfidf_weight, bm25_weight],['none', 'tf-idf', 'bm25_weight']):
        user_item_matrix = weight(user_item_matrix.T).T
        keys, values = zip(*param_grid.items())
        for v in itertools.product(*values):
            params = dict(zip(keys, v))
            prec = als_fit(user_item_matrix, params, result)
            params.update({'precision_at_k': prec, 'weight_method':method})
            score.append(params)
    score=sorted(score, key=lambda x:x['precision_at_k'], reverse=True)
    return score[0]

In [ ]:
param_grid = {'factors': [64, 128],
              'regularization': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5],
              'iterations': [5, 10, 15, 20],
              'calculate_training_loss': [True]}

In [ ]:
# Поиск выполняется от 1 часа 20 минут!!!
%%time

best_grid_search_results = grid_search(user_item_matrix, param_grid, result)

In [ ]:
best_grid_search_results

In [ ]:
import pandas as pd
import numpy as np


def prefilter_items(data, item_features=None, take_n_popular=5000):
    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')[
                     'user_id'].nunique().reset_index() / data[
                     'user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'},
                      inplace=True)

    top_popular = popularity[
        popularity['share_unique_users'] > 0.5].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 100]

    # Уберем товары, которые не продавались за последние 12 месяцев

    old_sales = data.groupby('item_id')['day'].max().reset_index()
    oldest = old_sales[
        old_sales['day'] > max(old_sales['day']) - 365].item_id.tolist()
    data = data[~data['item_id'].isin(oldest)]

    # Возьмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()

    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999


    return data

def postfilter_items(user_id, recommednations):
    pass